In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import sys
import time
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook as tqdm

sys.path.insert(0, "../../")

from rcalgo.tf.models.semi_supervised.mix_text import MixText, MixTextData
from rcalgo.tf.models.semi_supervised.mix_bert import MixBertModel
from rcalgo.tf.utils.tokenizers import CharTokenizer
from rcalgo.tf.training.hook import TensorInfoPrintHook
from rcalgo.tf.training.data.data_iterator import DataIterator
from rcalgo.tf.models.finetune.google_bert import BertClassificationModel
from rcalgo.tf.models.finetune.google_bert import BertIdSequenceClassificationModel
from rcalgo.tf.utils.preprocessors import BertSentencePreprocessor

In [2]:
# load data
data_dir = "/home/web_server/antispam/project/datasets/female_vulgar/"
train_data = pd.read_csv(data_dir + 'train_400.csv')
test_data = pd.read_csv(data_dir + 'test.csv')
# count word dict
tokenizer = CharTokenizer.build_from_corpus(test_data['text'], freq_threshold=2)
word_dict = tokenizer.word2idx

In [3]:
# augment unsupvised data with EDA
# warning: this step may be time-costing

org_unsup_file = data_dir + "unsup0.txt"
aug_unsup_file = data_dir + "unsup1.txt"

if not os.path.exists(aug_unsup_file):
    from rcalgo.tf.models.augmentation.token_level.eda import EDA
    eda = EDA(stopwords=[], need_cut=True, rand_seed=0)
    start_t = time.time()
    with open(org_unsup_file) as in_f, open(aug_unsup_file, "w") as out_f:
        sentences = [line.strip() for line in in_f]
        aug_sentences = eda.run(sentences, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, times=1)
        for line in aug_sentences:
            out_f.write(f"{line}\n")

    print(f"Unsup data augment time cost: {time.time() - start_t} s")

In [4]:
sup_texts = train_data['text']
labels = train_data['label']
batch_size = 32
unsup_ratio = 1
gpu_num = 2
num_epochs = 220
num_train_steps = num_epochs * (int(len(sup_texts)) // (batch_size * gpu_num))
print(f"num_train_steps: {num_train_steps}")

num_train_steps: 2640


In [5]:
# create execute model
config = {
    # mix text config
    "mix_type": "mix_sentence_hidden", 
    "lambda_u": 1,
    "lambda_u_hinge" : 1,
    "batch_size": batch_size,
    "epoch_num": num_epochs,
    "num_train_steps": num_train_steps,
    "T": 0.5,
    "unsup_ratio": unsup_ratio,
    "aug_num": 1,
    "gpu_id": 0,
    "gpu_num": gpu_num,
    
    # model config
    "max_seq_length": 128,
    "epochs": 4,
    "test_size": 0.1,
    "batch_size": 32,
    "warmup_proportion": 0.1,

    "learning_rate": 0.0001,

    "vocab_file": "/home/web_server/antispam/project/pretrained_models/"
                  "chinese_L-12_H-768_A-12/vocab.txt",
    "checkpoint_file": "/home/web_server/antispam/project/pretrained_models/"
                       "chinese_L-12_H-768_A-12/bert_model.ckpt"
}

In [6]:
test_sup_texts = test_data['text']
test_labels = test_data['label']

In [7]:
org_unsup_texts = []
aug_unsup_texts = []
with open(org_unsup_file) as org_f:
    all_org_unsup_texts = [line.strip() for line in org_f.readlines()]
with open(aug_unsup_file) as aug_f:
    all_aug_unsup_texts = [line.strip() for line in aug_f.readlines()]

unsup_size = len(all_org_unsup_texts) // unsup_ratio
for i in range(unsup_size):
    org_unsup_texts.append([all_org_unsup_texts[i + j] for j in range(unsup_ratio)])
    aug_unsup_texts.append([all_aug_unsup_texts[i + j] for j in range(unsup_ratio)])
    
test_size = len(test_labels)
test_org_unsup_texts = org_unsup_texts[:test_size]
test_aug_unsup_texts = aug_unsup_texts[:test_size]

In [8]:
print(f"sup_texts: ")
print(sup_texts[:5])
print(f"org_unsup_texts: ")
print(org_unsup_texts[:5])
print(len(org_unsup_texts))
print(f"aug_unsup_texts: ")
print(aug_unsup_texts[:5])
print(len(test_aug_unsup_texts))

sup_texts: 
0    祝雅儿新年快乐，开心快乐每一天，越来越漂亮哈[笑哭]
1                           非洲哥
2                         鬼街在哪啊
3                            哈卵
4                   差那点洗脸洗手的时间？
Name: text, dtype: object
org_unsup_texts: 
[['祝雅儿新年快乐，开心快乐每一天，越来越漂亮哈[笑哭]'], ['非洲哥'], ['鬼街在哪啊'], ['哈卵'], ['差那点洗脸洗手的时间？']]
400000
aug_unsup_texts: 
[['祝雅儿新年快乐，开心快乐每虉天，越逨越漂亮哈[笑哭]'], ['狒洲哥'], ['鬼劫在哪啊'], ['哈灤'], ['差貀点洗亷洗収的时间？']]
40000


In [9]:
# compute num_train_steps and num_warmup_steps
n_examples = len(sup_texts)
config["num_train_steps"] = int(n_examples * config["epochs"] / config["batch_size"] / gpu_num)
config["num_warmup_steps"] = int(config["num_train_steps"] * config["warmup_proportion"])
print(config)

{'mix_type': 'mix_sentence_hidden', 'lambda_u': 1, 'lambda_u_hinge': 1, 'batch_size': 32, 'epoch_num': 220, 'num_train_steps': 24, 'T': 0.5, 'unsup_ratio': 1, 'aug_num': 1, 'gpu_id': 0, 'gpu_num': 2, 'num_classes': 2, 'max_seq_length': 128, 'epochs': 4, 'test_size': 0.1, 'warmup_proportion': 0.1, 'learning_rate': 0.0001, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_warmup_steps': 2}


## training with original model

In [10]:
preprocessor = BertSentencePreprocessor(config["vocab_file"])
sup_texts_ids= [preprocessor.convert_text_to_ids_string(text) for text in sup_texts]
test_sup_texts_ids= [preprocessor.convert_text_to_ids_string(text) for text in test_sup_texts]

In [11]:
tf.compat.v1.reset_default_graph()
model = BertIdSequenceClassificationModel(config, distribute="tf_distribute")
model.build_model()
model.trainer.initialize()
model.trainer.load_single_graph_checkpoint(config["checkpoint_file"])
print(model.model_summary())

2020-11-17 15:50:28,138  model.py (73) (MainThread) : INFO  config: {'mix_type': 'mix_sentence_hidden', 'lambda_u': 1, 'lambda_u_hinge': 1, 'batch_size': 32, 'epoch_num': 220, 'num_train_steps': 24, 'T': 0.5, 'unsup_ratio': 1, 'aug_num': 1, 'gpu_id': 0, 'gpu_num': 2, 'num_classes': 2, 'max_seq_length': 128, 'epochs': 4, 'test_size': 0.1, 'warmup_proportion': 0.1, 'learning_rate': 0.0001, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_warmup_steps': 2}


Instructions for updating:
reduction_indices is deprecated, use axis instead



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



2020-11-17 15:50:36,573  single_worker_graph_copy_trainer.py (92) (MainThread) : INFO  Initializing trainer ...
2020-11-17 15:50:36,574  single_worker_graph_copy_trainer.py (105) (MainThread) : INFO  gpu_num: 2
2020-11-17 15:50:36,576  graph_copy.py (424) (MainThread) : INFO  build_train_graph kwargs: {'mix_type': 'mix_sentence_hidden', 'lambda_u': 1, 'lambda_u_hinge': 1, 'batch_size': 32, 'epoch_num': 220, 'num_train_steps': 24, 'T': 0.5, 'unsup_ratio': 1, 'aug_num': 1, 'num_classes': 2, 'max_seq_length': 128, 'epochs': 4, 'test_size': 0.1, 'warmup_proportion': 0.1, 'learning_rate': 0.0001, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_warmup_steps': 2, 'excluded_task_set': ['pretrained_model/default'], 'pretrained_model_scope': 'pretrained_model'}
2020-11-17 15:50:36,576  graph_copy.py (426) (MainThread) : INFO


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2020-11-17 15:50:49,723  graph_copy.py (448) (MainThread) : INFO  build_train_graph (bert_id_sequence_classification_model): building inputs...
2020-11-17 15:50:49,724  graph_copy.py (450) (MainThread) : INFO  task: bert_id_sequence_classification_model, <class 'str'>
2020-11-17 15:50:49,728  graph_copy.py (478) (MainThread) : INFO  build_train_graph (bert_id_sequence_classification_model): building per replica graph...


...

2020-11-17 15:51:20,416  graph_copy.py (486) (MainThread) : INFO  train_ops: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Adam_bert_id_sequence_classification_model/Identity:0' shape=() dtype=int64>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'Adam_bert_id_sequence_classification_model/Identity_1:0' shape=() dtype=int64>
}], losses: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Mean:0' shape=() dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Mean:0' shape=() dtype=float32>
}], metrics: [PerReplica:{
  0 /replica:0/task:0/device:GPU:0: <tf.Tensor 'Cast:0' shape=(?,) dtype=float32>,
  1 /replica:0/task:0/device:GPU:1: <tf.Tensor 'replica_1/Cast:0' shape=(?,) dtype=float32>
}]


INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).


2020-11-17 15:51:20,593  single_worker_graph_copy_trainer.py (119) (MainThread) : INFO  all_tasks: ['bert_id_sequence_classification_model']
2020-11-17 15:51:21,226  single_worker_graph_copy_trainer.py (122) (MainThread) : INFO  init variables with sess target: None
2020-11-17 15:51:41,005  single_worker_graph_copy_trainer.py (124) (MainThread) : INFO  single_worker: initialized
2020-11-17 15:51:41,006  single_worker_graph_copy_trainer.py (125) (MainThread) : INFO  Init time cost (secs):  64.432
2020-11-17 15:51:41,007  single_worker_graph_copy_trainer.py (532) (MainThread) : INFO  Reload single graph from /home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt


2020-11-17 15:51:41,012  single_worker_graph_copy_trainer.py (540) (MainThread) : INFO  Restore 199 vars from checkpoint.



total parameter number: 102269186
                                      variable_name variable_shape parameters
0                 bert/embeddings/word_embeddings:0   [21128, 768]   16226304
1           bert/embeddings/token_type_embeddings:0       [2, 768]       1536
2             bert/embeddings/position_embeddings:0     [512, 768]     393216
3                  bert/embeddings/LayerNorm/beta:0          [768]        768
4                 bert/embeddings/LayerNorm/gamma:0          [768]        768
..                                              ...            ...        ...
196  bert/encoder/layer_11/output/LayerNorm/gamma:0          [768]        768
197                      bert/pooler/dense/kernel:0     [768, 768]     589824
198                        bert/pooler/dense/bias:0          [768]        768
199                                output_weights:0       [2, 768]       1536
200                                   output_bias:0            [2]          2

[201 rows x 3 columns]


In [12]:
model.train_and_evaluate(
    [sup_texts_ids, labels],
    [test_sup_texts_ids, test_labels],
    batch_size=config["batch_size"],
    epochs=config["epochs"],
    hooks=[])

2020-11-17 15:06:33,361  single_worker_graph_copy_trainer.py (150) (MainThread) : INFO  tasks: ['bert_id_sequence_classification_model']
2020-11-17 15:06:33,362  single_worker_graph_copy_trainer.py (151) (MainThread) : INFO  loss: Tensor("Identity:0", shape=(), dtype=float32, device=/replica:0/task:0/device:CPU:0), loss_list: [<tf.Tensor 'Identity:0' shape=() dtype=float32>], task name: bert_id_sequence_classification_model
2020-11-17 15:06:33,363  single_worker_graph_copy_trainer.py (156) (MainThread) : INFO  do_test: True
2020-11-17 15:06:33,363  single_worker_graph_copy_trainer.py (157) (MainThread) : INFO  building dataset ...
2020-11-17 15:06:33,363  data.py (19) (MainThread) : INFO  Data: will drop 15 remainder_size data items.
2020-11-17 15:06:33,444  data.py (19) (MainThread) : INFO  Data: will drop 0 remainder_size data items.
2020-11-17 15:06:35,506  single_worker_graph_copy_trainer.py (174) (MainThread) : INFO  start training sess run ...


'epoch 0 train time(sec): 16.79695749282837'
{'best_train_metrics': [0.5182],
 'best_validate_metrics': [0.6791],
 'epoch_index': 0,
 'train_loss': [0.6952],
 'train_metrics': [0.5182],
 'validate_loss': [0.6056],
 'validate_metrics': [0.6791]}


  0%|          | 0/12 [00:00<?, ?it/s]                                                                                                    

'epoch 1 train time(sec): 3.33998441696167'
{'best_train_metrics': [0.8255],
 'best_validate_metrics': [0.6791],
 'epoch_index': 1,
 'train_loss': [0.4389],
 'train_metrics': [0.8255],
 'validate_loss': [0.7803],
 'validate_metrics': [0.6080]}


  0%|          | 0/12 [00:00<?, ?it/s]                                                                                                    

'epoch 2 train time(sec): 3.3070454597473145'
{'best_train_metrics': [0.8750],
 'best_validate_metrics': [0.7522],
 'epoch_index': 2,
 'train_loss': [0.3071],
 'train_metrics': [0.8750],
 'validate_loss': [0.7302],
 'validate_metrics': [0.7522]}


'epoch 3 train time(sec): 3.3315913677215576'
{'best_train_metrics': [0.9557],
 'best_validate_metrics': [0.7593],
 'epoch_index': 3,
 'train_loss': [0.1436],
 'train_metrics': [0.9557],
 'validate_loss': [0.7306],
 'validate_metrics': [0.7593]}


## training with MixText wrapper

In [13]:
preprocessor = BertSentencePreprocessor(config["vocab_file"])
sup_texts_ids= [preprocessor.convert_text_to_ids_string(text) for text in sup_texts]
test_sup_texts_ids= [preprocessor.convert_text_to_ids_string(text) for text in test_sup_texts]
org_unsup_texts_ids= [[preprocessor.convert_text_to_ids_string(text) for text in texts] 
                      for texts in org_unsup_texts]
test_org_unsup_texts_ids= [[preprocessor.convert_text_to_ids_string(text) for text in texts] 
                           for texts in test_org_unsup_texts]
aug_unsup_texts_ids= [[preprocessor.convert_text_to_ids_string(text) for text in texts] 
                      for texts in aug_unsup_texts]
test_aug_unsup_texts_ids= [[preprocessor.convert_text_to_ids_string(text) for text in texts] 
                           for texts in test_aug_unsup_texts]

In [14]:
tf.compat.v1.reset_default_graph()
model = MixBertModel(config=config, name="default", distribute="tf_distribute")
mix_text_model = MixText(config=config, execute_model=model, name="default", distribute="tf_distribute")

2020-11-17 15:53:02,695  model.py (73) (MainThread) : INFO  config: {'mix_type': 'mix_sentence_hidden', 'lambda_u': 1, 'lambda_u_hinge': 1, 'batch_size': 32, 'epoch_num': 220, 'num_train_steps': 24, 'T': 0.5, 'unsup_ratio': 1, 'aug_num': 1, 'gpu_id': 0, 'gpu_num': 2, 'num_classes': 2, 'max_seq_length': 128, 'epochs': 4, 'test_size': 0.1, 'warmup_proportion': 0.1, 'learning_rate': 0.0001, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_warmup_steps': 2}


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.


2020-11-17 15:53:02,865  model.py (73) (MainThread) : INFO  config: {'mix_type': 'mix_sentence_hidden', 'lambda_u': 1, 'lambda_u_hinge': 1, 'batch_size': 32, 'epoch_num': 220, 'num_train_steps': 24, 'T': 0.5, 'unsup_ratio': 1, 'aug_num': 1, 'gpu_id': 0, 'gpu_num': 2, 'num_classes': 2, 'max_seq_length': 128, 'epochs': 4, 'test_size': 0.1, 'warmup_proportion': 0.1, 'learning_rate': 0.0001, 'vocab_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/vocab.txt', 'checkpoint_file': '/home/web_server/antispam/project/pretrained_models/chinese_L-12_H-768_A-12/bert_model.ckpt', 'num_warmup_steps': 2}


In [15]:
mix_text_model.build_model()

print(mix_text_model.model_summary())
print(mix_text_model.loss)



Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
input_ids: Tensor("MixText_1/concat:0", shape=(?, ?), dtype=int32)



/root/anaconda3/envs/tf1.15/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


total parameter number: 102269186
                                         variable_name variable_shape  \
0            MixText/bert/embeddings/word_embeddings:0   [21128, 768]   
1      MixText/bert/embeddings/token_type_embeddings:0       [2, 768]   
2        MixText/bert/embeddings/position_embeddings:0     [512, 768]   
3             MixText/bert/embeddings/LayerNorm/beta:0          [768]   
4            MixText/bert/embeddings/LayerNorm/gamma:0          [768]   
..                                                 ...            ...   
196  MixText/bert/encoder/layer_11/output/LayerNorm...          [768]   
197                 MixText/bert/pooler/dense/kernel:0     [768, 768]   
198                   MixText/bert/pooler/dense/bias:0          [768]   
199                           MixText/output_weights:0       [2, 768]   
200                              MixText/output_bias:0            [2]   

    parameters  
0     16226304  
1         1536  
2       393216  
3          768  
4   

In [24]:
logging_hook = TensorInfoPrintHook({"unsup_loss": mix_text_model.unsup_loss,
                                    "sup_loss": mix_text_model.sup_loss, 
                                    "sup_ce_loss": mix_text_model.sup_ce_loss,
                                    "sup_kl_loss": mix_text_model.sup_kl_loss,
                                    "consistency_loss": mix_text_model.consistency_loss,
                                    "self_training_loss": mix_text_model.self_training_loss})

In [27]:
train_data = MixTextData([sup_texts, org_unsup_texts, aug_unsup_texts, labels],
                     mode="train", batch_size=batch_size * config['gpu_num'], 
                     drop_remainder=True, name="train_data")

test_data = MixTextData([test_sup_texts, test_org_unsup_texts, test_aug_unsup_texts, test_labels],
                     mode="test", batch_size=batch_size * config['gpu_num'], 
                    drop_remainder=True, name="test_data")

2020-11-17 16:01:43,682  data.py (19) (MainThread) : INFO  train_data: will drop 15 remainder_size data items.
2020-11-17 16:01:46,444  data.py (19) (MainThread) : INFO  test_data: will drop 0 remainder_size data items.


In [28]:
mix_text_model.train_and_evaluate(train_data, test_data,
                batch_size=batch_size * config['gpu_num'],
                epochs=num_epochs,
                hooks=[logging_hook])

2020-11-17 16:01:47,434  single_worker_graph_copy_trainer.py (150) (MainThread) : INFO  tasks: ['default']
2020-11-17 16:01:47,434  single_worker_graph_copy_trainer.py (151) (MainThread) : INFO  loss: Tensor("Identity:0", shape=(), dtype=float32, device=/replica:0/task:0/device:CPU:0), loss_list: [<tf.Tensor 'Identity:0' shape=() dtype=float32>], task name: default
2020-11-17 16:01:47,435  single_worker_graph_copy_trainer.py (156) (MainThread) : INFO  do_test: True
2020-11-17 16:01:47,435  single_worker_graph_copy_trainer.py (157) (MainThread) : INFO  building dataset ...


sup_data_size: 384, unsup_data_size: 399985


2020-11-17 16:01:49,852  single_worker_graph_copy_trainer.py (174) (MainThread) : INFO  start training sess run ...


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[16384,3072] and type float on /job:localhost/replica:0/task:0/device:GPU:1 by allocator GPU_1_bfc
	 [[node replica_1/MixText_1/bert/encoder/layer_3/intermediate/dense/MatMul (defined at /root/anaconda3/envs/tf1.15/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[16384,3072] and type float on /job:localhost/replica:0/task:0/device:GPU:1 by allocator GPU_1_bfc
	 [[node replica_1/MixText_1/bert/encoder/layer_3/intermediate/dense/MatMul (defined at /root/anaconda3/envs/tf1.15/lib/python3.6/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_

## Export and infer

In [16]:
mix_text_model.export_freeze_graph("freeze", 1, multi_replica_ckpt_path="saved_models/best_model/default-epoch204.ckpt")


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 9 variables.
INFO:tensorflow:Converted 9 variables to const ops.


In [17]:
def load_feeze_graph(model_fn):
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
    config = tf.compat.v1.ConfigProto(gpu_options=gpu_options, allow_soft_placement=True)
    session = tf.compat.v1.Session(config=config, graph=tf.Graph())
    with session.graph.as_default():
        tf.compat.v1.train.import_meta_graph(model_fn)
        session.run(tf.compat.v1.tables_initializer())
    return session

sess = load_feeze_graph('freeze/1/frozen.pb')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [18]:
with sess.graph.as_default():
    inputs = tf.get_collection('input_dict')
    outputs = tf.get_collection('output_dict')[0]

    data_iter = DataIterator([test_sup_texts], 
                             np.array(range(len(test_sup_texts))), batch_size=batch_size)
    data_iter_tqdm = tqdm(data_iter, leave=False)
    res = []
    for idx, batch in data_iter_tqdm:
        feed_data = dict(zip(inputs, batch))
        batch_res = sess.run(outputs, feed_dict=feed_data)
        res.append(batch_res)
       
    np_res = np.concatenate(res, axis=0)
    
print(np_res)

/root/anaconda3/envs/tf1.15/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


[[0.3495476  0.65045244]
 [0.6364067  0.36359325]
 [0.03411315 0.9658869 ]
 ...
 [0.68831915 0.31168085]
 [0.08680675 0.9131933 ]
 [0.5116102  0.48838973]]


In [19]:
pos_predicts = np.argmax(np_res, axis=1)
pos_acc = accuracy_score(test_labels, pos_predicts)
print(f"acc on test: {pos_acc}")

acc on test: 0.701875
